In [4]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import csv

In [5]:

teamQualified = {'6':'India'}
teamResultTally = {'India':0,'Australia':0}
urls = []
results = []
for i,j in teamQualified.items():
    url = f"https://www.espncricinfo.com/team/{j}-{i}/match-schedule-fixtures-and-results"
    urls.append(url)


for u in urls:
        response = requests.get(u)
        htmlcontent = response.content
        soup = BeautifulSoup(htmlcontent,"html.parser")
        
#         print(u)
        for a in soup.find_all('div',attrs={'class':'ds-text-compact-xxs'}):
            result = soup.find_all('p',attrs = {'class':'ds-text-tight-s ds-font-regular ds-line-clamp-2 ds-text-typo-title'})

for r in result: 
                print(r.text)


India won by 7 wickets (with 185 balls remaining)
WA XI won by 36 runs
India won by 6 runs
Match abandoned without a ball bowled
India won by 4 wickets (with 0 balls remaining)
India won by 56 runs
South Africa won by 5 wickets (with 2 balls remaining)
India won by 5 runs (DLS method)
India won by 71 runs
(Reserve day available) - England won by 10 wickets (with 24 balls remaining)
Match abandoned without a ball bowled
India won by 65 runs
Match tied (DLS method)
New Zealand won by 7 wickets (with 17 balls remaining)
No result
No result
Bangladesh won by 1 wicket (with 24 balls remaining)
Bangladesh won by 5 runs
India won by 227 runs
India won by 188 runs
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to begin
Match yet to

In [6]:
players = []
ranking = []
results = []

url = "http://www.relianceiccrankings.com/ranking/odi/batting/"

response = requests.get(url)
htmlcontent = response.content
soup = BeautifulSoup(htmlcontent,"html.parser")
        
for a in soup.find_all('tr'):
        player = soup.find_all('a',attrs={'class':'players'})
        c= 0
        for p in player:
            players.append(p.text)
            c+=1
            ranking.append(str(c))
        break

results.append(ranking)
results.append(players)

res = list(zip(*results))

for r in res:
    print(r)

('1', ' Babar Azam')
('2', ' Imam-ul-Haq')
('3', 'Rassie van der Dussen')
('4', 'Quinton de Kock')
('5', 'David Warner')
('6', 'Steve Smith')
('7', 'Jonny Bairstow')
('8', 'Virat Kohli')
('9', 'Rohit Sharma')
('10', 'Kane Williamson')
('11', ' Fakhar Zaman')
('12', 'Shai Hope')
('13', 'Joe Root')
('14', 'Paul Stirling')
('15', 'Shreyas Iyer')
('16', ' Tamim Iqbal')
('17', 'Tom Latham')
('18', 'Jason Roy')
('19', 'David Miller')
('20', ' Mushfiqur Rahim')
('21', 'Shikhar Dhawan')
('22', 'Aaron Finch')
('23', 'Alex Carey')
('24', 'Martin Guptill')
('25', 'Glenn Maxwell')
('26', ' Rahmat Shah')
('27', 'Harry Tector')
('28', 'Nicholas Pooran')
('29', 'Travis Head')
('30', 'Liton Liton Das')
('31', 'Jos Buttler')
('32', ' Sikandar Raza')
('33', 'Kariyawasa Asalanka')
('34', ' Shakib Al Hasan')
('35', ' Mahmudullah')
('36', 'Gerhard Erasmus')
('37', ' Ishan Kishan')
('38', 'Lokesh Rahul')
('39', 'Shubman Gill')
('40', 'Calum MacLeod')
('41', ' Najibullah Zadran')
('42', 'Kyle Coetzer')
('43'

In [7]:
teams = []
ranking = []
results = []

url = "https://www.icc-cricket.com/rankings/mens/team-rankings/odi"

response = requests.get(url)
htmlcontent = response.content
soup = BeautifulSoup(htmlcontent,"html.parser")
        
for a in soup.find_all('tr'):
        team = soup.find_all('span',attrs={'class':'u-hide-phablet'})
        c = 0
        for t in team:
            c+=1
            teams.append(t.text)
            ranking.append(str(c))
        break

results.append(ranking)
results.append(teams)

res = list(zip(*results))

for r in res:
    print(r)

('1', 'New Zealand')
('2', 'England')
('3', 'Australia')
('4', 'India')
('5', 'Pakistan')
('6', 'South Africa')
('7', 'Bangladesh')
('8', 'Sri Lanka')
('9', 'Afghanistan')
('10', 'West Indies')
('11', 'Ireland')
('12', 'Scotland')
('13', 'Zimbabwe')
('14', 'Namibia')
('15', 'Netherlands')
('16', 'Oman')
('17', 'UAE')
('18', 'United States')
('19', 'Nepal')
('20', 'Papua New Guinea')


In [8]:
# India, New Zealand, England, Australia, Bangladesh, Pakistan, Afghanistan

urls = []
for i in range(2018,2023):
    url = f"https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id={i};type=year"
    urls.append(url)

matches = []
winners = []


dict = {'Team':['India','Australia','New Zealand','England','Bangladesh','Pakistan','Afghanistan'],
        'Total Matches':[0,0,0,0,0,0,0],
        'Matches Won':[0,0,0,0,0,0,0],
        'Winning Percentage':[0,0,0,0,0,0,0]
       }

for u in urls:

        response = requests.get(u)
        htmlcontent = response.content
        soup = BeautifulSoup(htmlcontent,"html.parser")
        
        for row in soup.findAll('table')[0].tbody.findAll('tr'):
            
            first_column = row.findAll('td')[0].contents
            for f1 in first_column: 
                matches.append(f1.text)
            second_column = row.findAll('td')[1].contents
            for f2 in second_column: 
                matches.append(f2.text)
            third_column = row.findAll('td')[2].contents
            for f3 in third_column: 
                winners.append(f3.text)
        
            


for i in matches:
    for j in range(len(dict['Team'])):
        if dict['Team'][j]==i:
            dict['Total Matches'][j]+= 1

            
for i in winners:
    for j in range(len(dict['Team'])):
        if dict['Team'][j]==i:
            dict['Matches Won'][j]+= 1
            
for i in range(len(dict['Team'])):
    dict['Winning Percentage'][i] += round((dict['Matches Won'][i]/dict['Total Matches'][i])*100,2)
            
            
new = pd.DataFrame.from_dict(dict)
# print(new['Team'])

team = [t for t in new['Team']] #list=list(new)
rank = []
for r in range(len(res)):
    rank.append([res[r][0],res[r][1]])

new['International Ranking'] = [1,1,1,1,1,1,1]
#print(new)
print(rank)


[['1', 'New Zealand'], ['2', 'England'], ['3', 'Australia'], ['4', 'India'], ['5', 'Pakistan'], ['6', 'South Africa'], ['7', 'Bangladesh'], ['8', 'Sri Lanka'], ['9', 'Afghanistan'], ['10', 'West Indies'], ['11', 'Ireland'], ['12', 'Scotland'], ['13', 'Zimbabwe'], ['14', 'Namibia'], ['15', 'Netherlands'], ['16', 'Oman'], ['17', 'UAE'], ['18', 'United States'], ['19', 'Nepal'], ['20', 'Papua New Guinea']]


In [9]:
new1=[]
for i in new['Team']:
    for j in range(len(rank)):
        if i==rank[j][1]:
            new1.append(rank[j][0])
print(new1)
new['International Ranking'] = new1
print(new)

['4', '3', '1', '2', '7', '5', '9']
          Team  Total Matches  Matches Won  Winning Percentage  \
0        India             87           54               62.07   
1    Australia             69           37               53.62   
2  New Zealand             57           37               64.91   
3      England             76           46               60.53   
4   Bangladesh             68           41               60.29   
5     Pakistan             61           29               47.54   
6  Afghanistan             55           27               49.09   

  International Ranking  
0                     4  
1                     3  
2                     1  
3                     2  
4                     7  
5                     5  
6                     9  


In [23]:
new.to_csv('cricket.csv')

In [24]:
df=spark.createDataFrame(new)

In [25]:
df.show()

Py4JJavaError: An error occurred while calling o82.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (192.168.217.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(

In [26]:
sc=spark.sparkContext

In [27]:
myRDD=sc.parallelize(new)

AttributeError: 'RDD' object has no attribute 'show'

In [17]:
column=['Team','Total Matches','Matches Won','Winning Percentage','International Ranking'] 

In [18]:
mydf=myRDD.toDF(column)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.217.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.schedu

In [10]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [12]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
import pyspark
#sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [35]:
print(4+5)

9


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [28]:
import spark
 
df = spark.createDataFrame()
df.printSchema()


AttributeError: module 'spark' has no attribute 'createDataFrame'

In [10]:
import pyspark

In [38]:
df=pd.DataFrame(new)
print(df)


          Team  Total Matches  Matches Won  Winning Percentage  \
0        India             87           54               62.07   
1    Australia             69           37               53.62   
2  New Zealand             57           37               64.91   
3      England             76           46               60.53   
4   Bangladesh             68           41               60.29   
5     Pakistan             61           29               47.54   
6  Afghanistan             55           27               49.09   

  International Ranking  
0                     4  
1                     3  
2                     1  
3                     2  
4                     7  
5                     5  
6                     9  


In [14]:
df=spark.read.csv('cricket.csv',header=True)

AttributeError: module 'pyspark' has no attribute 'read'